In [1]:
from config import *
from tools import *
from U_Resnet import *

In [2]:
img_path = explore_dir("./dataset/val_images/")[1][:,np.newaxis]
mask_path = explore_dir("./dataset/TrainVal_parsing_annotations/val_segmentations/")[1][:,np.newaxis]
val_paths = np.concatenate((img_path,mask_path), axis=1)

val_dataset=np.array(Pool(processes=4).map(im_and_roi_read,val_paths)).transpose(1,0,2,3,4)
val_img, val_roi = val_dataset
val_data, val_label = separate_batches(val_img,val_roi, 20)

tf.reset_default_graph()

uresnet = U_Resnet((256,256,3), (256,256,3),20)

saver = tf.train.Saver()
saver.restore(uresnet.sess, "./snapshot/U_Resnet.ckpt")

INFO:tensorflow:Restoring parameters from ./snapshot/U_Resnet.ckpt


In [ ]:
with tqdm(total=len(val_data)) as pbar:
    pbar.set_description("[ Estimating Validation Dataset ]")
    roi_result = []
    for v in range(len(val_data)):
        result = uresnet.sess.run(uresnet.score, feed_dict={uresnet.x: val_data[v],uresnet.keep_prob: 1.0})
        result = np.argmax(result,axis=3)[:,:,:,np.newaxis]
        result = masking(result)
        roi_result += list(result)
        pbar.update(1)

[ Estimating Validation Dataset ]:  98%|█████████▊| 489/500 [21:47<00:56,  5.11s/it]

In [ ]:
idx=1
fig, plot = plt.subplots(1,3)
fig.set_size_inches(15, 15)
plot[0].imshow(val_img[idx][:,:,[2,1,0]])
plot[0].set_title("Original Image")
plot[1].imshow(roi_result[idx][:,:,[2,1,0]])
plot[1].set_title("Predicted RoI Mask")
plot[2].imshow(masking(val_roi[idx][np.newaxis,:,:,0])[0])
plot[2].set_title("Ground Truth RoI Mask")